## Descripción del algoritmo


Se ha desarrollado un código en Python 3 a partir de los modelos globales. Se pretende comprobar si los resultados de la simulación son compatibles con datos disponibles de generación de un campo solar real.
Se ha seguido el paradigma de Programación Orientada a Objetos, de tal forma que la planta solar y sus diferentes subsistemas se modelan mediante diferentes Clases. Estas clases disponen de métodos para la entrada/salida de información, así como atributos necesarios para caracterizar cada Objeto.
En la descripción de las clases se seguirá un proceso ascendente en la estructura constructiva del campo solar, comenzando con los objetos más elementales, los HCE, y terminando con el campo solar en su conjunto. Además, otras clases se desarrollan, aunque de una forma muy elemental, para permitir futuras integraciones con otros elementos de la planta (Bloque de Potencia, Tren de Generación, Sistema BOP, etc.) 

### Clase HCE (Heat Collector Element)

Físicamente, un HCE es un tubo de acero con una envolvente de vidrio de tal forma que entre el tubo de acero y la envolvente queda un espacio en el que se ha practicado el vacío. Por el interior del tubo circula el fluido caloportador (HTF, Heat Transfer Fluid) que aumenta su energía térmica al recibir el calor procedente de las paredes interiores del tubo.
El tubo recibe durante el proceso de fabricación, un recubrimiento selectivo que mejora sus propidades físicas para absorber la raciación solar. 
De cara a modelar el funcionamiento del HCE como elemento responsable de calentar el HTF, se define la clase HCE que consta de los siguientes atributos:

* Temperatura de entrada del HTF, tin
* Presión de entrada del HTF, pin
* Caudal másico del HTF, massflow
* Potencia calorífica absorbida, qabs
* Potencia calorífica perdida, qperd
* Rendimiento global del HCE, pr
* Temperatura de salida del HTF, tout
* Presión de salida del HTF, pout


Con estos parámetros el comportamiento del HCE queda totalmente caracterizado en el sistema desde el punto de vista del proceso de generación. No todos estos atributos (pueden entenderse como variables) son independientes entre sí. Por ejemplo, el rendimiento es:

$ pr = 1 - q_{perd} / q_{abs} $

Más adelante veremos como un objeto (instancia) de la clase HCE puede ser procesada por otra instancia de la clase Modelo para modelar su comportamiento según el modelo teórico que se pretende validar.

La Clase HCE también nos proporciona algunos métodos necesarios para el trabajo de procesamiento de la información, asignación y recuperación de valores de los atributos.

Otro asepecto importante es que cada instancia de la clase HCE tiene un atributo de tipo 'diccionario', en el que a modo de lista de pares clave-valor va a recibir aquellos parámetros que posteriormente serán empleados por el Modelo. Los diferentes autores que han elaborado modelos para los HCE no siempren utilizan los mismos parámetros ni idénticos identificadores. Al emplear un diccionario se facilita la tarea de implementación de nuevos Modelos, sin que sea necesario cambiar los atributos de la clase en cada ocasión.

Finalmente, un HCE es un elemento que ocupa una determinada posición dentro del SCA (Solar Collector Assembly). Más adelante se verá que, para determinadas simulaciones, el orden que ocupa dentro de la fila de HCEs y el propio SCA al que pertenece, pueden ser datos necesarios a la hora de estudiar su comportamiento. Por este motivo cada HCE mantiene una referencia al SCA al que pertenece y guarda información sobre su posición relativa dentro de él.






In [ ]:
class HCE(object):

    def __init__(self, sca, hce_order, settings):

        self.sca = sca
        self.hce_order = hce_order
        self.parameters = dict(settings)
        self.tin = 0.0
        self.tout = 0.0
        self.pr = 0.0
        self.qabs = 0.0
        self.qperd = 0.0
        self.pout = 0.0
        self.pin = 0.0

    def set_tin(self):

        if self.hce_order > 0:
            self.tin = self.sca.hces[self.hce_order-1].tout
        elif self.sca.sca_order > 0:
            self.tin = self.sca.loop.scas[self.sca.sca_order-1].hces[-1].tout
        else:
            self.tin = self.sca.loop.tin

    def set_pin(self):

        if self.hce_order > 0:
            self.pin = self.sca.hces[self.hce_order-1].pout
        elif self.sca.sca_order > 0:
            self.pin = self.sca.loop.scas[self.sca.sca_order-1].hces[-1].pout
        else:
            self.pin = self.sca.loop.pin

    def set_tout(self, htf):

        HL = htf.get_deltaH(self.tin, self.sca.loop.pin)

        if self.qabs > 0:

            h = (np.pi * self.parameters['Absorber tube outer diameter'] *
                 self.parameters['Length'] *
                 self.qabs * self.pr / self.sca.loop.massflow)

        else:
            h = -self.qperd / self.sca.loop.massflow

        self.tout = htf.get_T(HL + h, self.sca.loop.pin)


    def set_pout(self, htf):

        # TO-DO CÁLCULLO PERDIDA DE CARGA:
        # Ec. Colebrook-White para el cálculo del factor de fricción de Darcy

        re_turbulent = 4000

        k = self.parameters['Inner surface roughness']
        D = self.parameters['Absorber tube inner diameter']
        re = htf.get_Reynolds(D, self.tin, self.pin, self.sca.loop.massflow)


        if re < re_turbulent:
            darcy_factor = 64 / re

        else:
            # a = (k /  D) / 3.7
            a = k / 3.7
            b = 2.51 / re
            x = 1

            fx = lambda x: x + 2 * np.log10(a + b * x )

            dfx = lambda x: 1 + (2 * b) / (np.log(10) * (a + b * x))

            root = sc.optimize.newton(fx,
                                      x,
                                      fprime=dfx,
                                      maxiter=10000)

            darcy_factor = 1 / (root**2)


        rho = htf.get_density(self.tin, self.pin)
        v = 4 * self.sca.loop.massflow / (rho * np.pi * D**2)
        g = sc.constants.g

        # Ec. Darcy-Weisbach para el cálculo de la pérdida de carga
        deltap_mcl = darcy_factor * (self.parameters['Length'] / D ) * (v**2 / (2 * g))

        deltap = deltap_mcl * rho * g

        self.pout = self.pin - deltap

    def set_krec(self, t):

        self.parameters['krec'] = (0.0153)*(t - 273.15) + 14.77

    def get_previous(self):

        return self.sca.hces[self.hce_order-1]

    def get_index(self):

        if hasattr(self.sca.loop, 'subfield'):
            index = [self.sca.loop.subfield.name,
                self.sca.loop.loop_order,
                self.sca.sca_order,
                self.hce_order]
        else:
            index = ['prototype',
                self.sca.loop.loop_order,
                self.sca.sca_order,
                self.hce_order]

        return index

    def get_pr_opt_peak(self, aoi, solarpos, row):

        alpha = self.get_absorptance()
        tau = self.get_transmittance()
        rho = self.sca.parameters['Reflectance']
        gamma = self.sca.get_solar_fraction(aoi, solarpos, row)

        pr_opt_peak = alpha * tau * rho * gamma

        if pr_opt_peak > 1 or pr_opt_peak < 0:
            print("ERROR", pr_opt_peak)

        return pr_opt_peak


    def get_pr_geo(self, aoi, solarpos, row):

        if aoi > 90:
            pr_geo = 0.0

        else:
            # Llamado "bordes" en Tesis. Pérdidas de los HCE de cabecera según aoi
            sca_unused_length = (self.sca.parameters["Focal Len"] *
                                 np.tan(np.radians(aoi)))

            unused_hces = sca_unused_length // self.parameters["Length"]

            unused_part_of_hce = ((sca_unused_length % self.parameters["Length"]) /
                                  self.parameters["Length"])

            if self.hce_order < unused_hces:
                pr_geo = 0.0

            elif self.hce_order == unused_hces:
                pr_geo = ((sca_unused_length % self.parameters["Length"]) /
                                  self.parameters["Length"])
            else:
                pr_geo = 1.0

            # pr_geo = 1- (self.sca.parameters["Focal Len"] * np.tan(np.radians(aoi)) /
            #              (len(self.sca.hces) * self.parameters["long"]))

            if pr_geo > 1.0 or pr_geo < 0.0:
                print("ERROR pr_geo out of limits", pr_geo)


        return pr_geo


    def get_pr_shadows(self, aoi, solarpos, row):

        # Llamado "sombras" en Tesis. Pérdidas por sombras. ¿modelar sobre el SCA?
        # Sombras debidas a otros lazos

        shadowing = (1 -
                     np.sin(np.radians(abs(solarpos['elevation'][0]))) *
                     self.sca.loop.row_spacing /
                     self.sca.parameters['Aperture'])

        if shadowing < 0.0:
            shadowing = 0.0

        if solarpos['elevation'][0] < 0:
            shadowing = 1.0

        pr_shadows = 1 - shadowing

        if  pr_shadows > 1 or  pr_shadows < 0:
            print("ERROR",  pr_shadows)

        return pr_shadows


    def get_hext(self, wspd):

        #  TO-DO:

        return 0.0


    def get_emittance(self, tro, wspd):


        #  Eq. 5.2 Barbero
        eext = (self.parameters['Absorber emittance factor A0'] +
                self.parameters['Absorber emittance factor A1'] *
                (tro - 273.15))
        """
        Lineal Increase if wind speed lower than 4 m/s up to 1% at 4 m/s
        Lineal increase over 4 m/s up to 2% at 7 m/s
        """
        if wspd <4:
            eext = eext * (1 + 0.01 * wspd / 4)

        else:
            eext = eext * (1 + 0.01 * (0.3333 * wspd - 0.3333))

        return eext


    def get_absorptance(self):

        #
        alpha = self.parameters['Absorber absorptance']

        return alpha


    def get_transmittance(self):

        #dependiendo si hay vídrio y si hay vacío
        tau = self.parameters['Envelope transmittance']

        return tau


    def get_reflectance(self):

        return self.sca.parameters['Reflectance']

### Clase SCA (Solar Collector Assambly)

Un SCA es una estructura compuesta por una serie de reflectores que cocentran la radiación solar sobre los HCE. Desde el punto de vista operativo, un SCA cuenta con capacidad de movimiento independiente respecto al resto de SCAs de la planta, por lo que es la unidad mínima de control de enfoque o desenfoque de la radiación solar en el campo solar. La clase SCA nos permite modelar cada SCA teniendo en cuenta las propiedades  de los reflectores (reflectividad, suciedad de los espejos, precisión del movimiento de seguimiento solar, etc.) 

En plantas de colectores cilindroparabólicos lo más frecuente es que el sistema de seguimiento tenga su eje de rotación alineado en la dirección Norte-Sur, con el fin de hacer un seguimiento Este-Oeste de la trayectoria solar a lo largo del día. No obstante, una configuración coneje Este-Oeste también puede ser interesante en algunos casos y el modelo también permite esta configuración.

Dentro del campo solar, cada HCE debe pertenecer a un SCA. El primer HCE del SCA recibe el fluido caloportadador procedente de otro SCA o de las tuberías colectoras de HTF frio. El último HCE del SCA entrega el HTF más caliente al siguiente SCA o a las tuberías colectoras de HTF caliente. 


In [1]:
class SCA(object):

# Uso de __slots__ permite ahorrar memoria RAM
#    __slots__= ['SCA_configuration']
    def __init__(self, loop, sca_order, settings):

        self.loop = loop
        self.sca_order = sca_order
        self.hces = []
        self.status = 'focused'
        self.tracking_angle = 0.0
        self.parameters = dict(settings)
        self.surface_tilt = 0.0
        self.surface_azimuth = 180.0


    def get_solar_fraction(self, aoi, solarpos, row):

        if self.status == 'defocused':
            solarfraction = 0.0
        elif self.status == 'focused':

            # faltaría: factor de forma del Sol, dependiente de solpos, geometría
            # absorbedor, sca y atmósfera.
            # Factor: se puede usar para tasa de espejos rotos, por ejemplo
            # Availability: podría ser un valor binario 0 o 1

            solarfraction = (self.parameters['Reflectance'] *
                             self.parameters['Geom.Accuracy'] *
                             self.parameters['Track Twist'] *
                             self.parameters['Cleanliness'] *
                             self.parameters['Dust'] *
                             self.parameters['Factor'] *
                             self.parameters['Availability'] *
                             self.get_sca_shadows(aoi, solarpos, row))
        else:
            solarfraction = 1.0

        if  solarfraction > 1 or  solarfraction < 0:
            print("ERROR",  solarfraction)

        return solarfraction

    def get_sca_shadows(self, aoi, solarpos, row):
        
        # estas pérdidas las he traslado al Lazo

        # Pérdidas por sombras en el sca:
        # - Sombras de otros SCA. Cálculo geometrico a partir de la distancia entre lazos.
        # - Nubes

        pr_shadows = 1.0

        return pr_shadows


    def get_IAM(self, theta):

        F0 = self.parameters['IAM Coefficient F0']
        F1 = self.parameters['IAM Coefficient F1']
        F2 = self.parameters['IAM Coefficient F2']

#        PROVISIONAL HASTA ACLARAR LA FÓRMULA DE IAM
#        kiam = (1-2.23073e-4 * theta - 1.1e-4 * theta**2 +
#                3.1859e-6 * theta**3 - 4.85509e-8 * theta** 4)
#        return kiam


        if (theta > 0 and theta < 80):
            theta = np.radians(theta)
            kiam = F0 + (F1*theta+F2*theta**2) / np.cos(theta)

            if kiam > 1:
                kiam = 2 -kiam

        else:
            kiam = 0.0

        if  kiam > 1 or  kiam < 0:
            print("ERROR",  kiam, theta)

        return kiam


    def get_aoi(self, solarpos):

        sigmabeta = 0.0
        beta0 = 0.0

        if self.parameters['Tracking Type'] == 1: # N-S single axis tracker
            if solarpos['azimuth'][0] > 0 and solarpos['azimuth'][0] <= 180:
                surface_azimuth = 90 # Surface facing east
            else:
                surface_azimuth = 270 # Surface facing west
        elif self.parameters['Tracking Type'] == 2:  # E-W single axis tracker
            surface_azimuth = 180  # Surface facing the equator

        beta0 = np.degrees(
                    np.arctan(np.tan(np.radians(solarpos['zenith'][0])) *
                              np.cos(np.radians(surface_azimuth -
                                                solarpos['azimuth'][0]))))

        if beta0 >= 0:
            sigmabeta = 0
        else:
            sigmabeta = 1

        beta = beta0 + 180 * sigmabeta
        aoi = pvlib.irradiance.aoi(beta,
                                   surface_azimuth,
                                   solarpos['zenith'][0],
                                   solarpos['azimuth'][0])
        return aoi

### Clase Loop y Clase PrototypeLoop

Un Loop o lazo es un cojunto de SCA conectados en serie, de tal forma que el HTF que entra frío al lazo experimenta un salto térmico cuando transita por él. El sistema de control ajusta el estado de enfoque o desenfoque de cada SCA en el lazo con el fin de conseguir que la temperatura de salida sea la de consigna. Por motivos de económicos, el caudal de HTF no suele ser regulable a nivel de lazo, pues obligaría a instalar una válvula de control en cada uno de ellos y por tanto todos los lazos de un mismo subcampo suelen tener un caudal muy parecido. Se ha desarrollado también una Clase Subfield para dar cuenta del conjunto de lazos que pertenecen a un mismo subcampo y, por tanto, pueden variar su caudal de forma independiente de los lazos de otro subcampo. En un campo solar suele haber haber varios subcampos que pueden regular su caudal independientemente. Los casos más extremos serían el de un campo solar con un único subcampo, en el que todos los lazos pertenecen al mismo subcampo y, en el otro extremo, un campo solar con tantos subcampos como lazos tiene, de tal forma que cada lazo es el único en su subcampo, y por tanto cada lazo puede regular su caudal independientemente.

Cada lazo del campo solar es modelado mediante una instancia de la clase Loop. Cada instancia mantiene referencias al subcampo al que pertenece y a los SCA que contiene. Atributos importantes de estas instancias serán el caudal de HTF en el lazo, las temperaturas y presiones de entrada y salidad del HTF y el rendimiento completo del lazo.

El código permite trabajar de dos formas, bien calculando y ajustando el caudal requerido para conseguir una temperatura de HTF determinada o bien fijando el caudal de HTF y calculando qué temperatura de salida tendría el HTF. En caso de que esta tempertura de salida supere el máximo permitido (un valor configurable) el código considera que se producirá un desenfoque en el SCA en que se alcance esta temperatura y el HTF dejará de calentarse. En este caso se suele decir que se produce un vertido de energía o desaprovechamiento de la radiación existente. El código permite contabilizar esta energía desaprovechada por cada lazo durante estas situaciones.

La Clase PrototypeLoop hereda de la Clase Loop sus principales métodos y atributos pero supone una pequeña variación de una instancia o lazo definido por Loop ya que se trata de un lazo "típico" o "promedio", que presenta una configuración constructiva idéntica  a la del resto de lazos de la planta pero no pertenece a ningún subcampo solar. Este lazo especial o prototipo se empleará cuando queramos realizar un estudio paramétrico del comportamiento del lazo, no de la planta, y también para realizar una simulación mucho más rápida cuando asumamos la hipótesis de que todos los lazos de la planta se comportan de igual manera. Esta aproximación es la que se hace en aplicaciones como SAM, donde no se simulan todos los lazos para modelar el campo solar, sino que se simula solo un lazo y el resultado se obtiene multiplicando el caudal de salida de este lazo por el número de lazos que forman la planta.



In [7]:
class Loop(object):

    def __init__(self, solarfield, loop_order, subfield = None):

        self.solarfield = solarfield
        self.subfield = subfield
        self.scas = []
        self.loop_order = loop_order

        self.tin = 0.0
        self.tout = 0.0
        self.pin = 0.0
        self.pout = 0.0
        self.tmax = 0.0
        self.massflow = 0.0
        self.req_massflow = 0.0  # Required massflow (in order to achieve the setpoint tout)
        self.pr_req_massflow = 0.0
        self.pr_act_massflow = 0.0

        self.act_tin = 0.0
        self.act_tout = 0.0
        self.act_pin = 0.0
        self.act_pout = 0.0
        self.act_massflow = 0.0  # Actual massflow measured by the flowmeter

        # self.rated_tin = 0.0
        # self.rated_tout = 0.0
        # self.rated_pin = 0.0
        # self.rated_pout = 0.0
        # self.rated_massflow = 0.0

        self.wasted_power = 0.0
        self.row_spacing = self.solarfield.solarfield_settings['row_spacing']

        self.tracking = True

    def initialize(self):

        self.massflow = self.subfield.massflow / len(self.subfield.loops)
        self.tin = self.subfield.tin
        self.pin = self.subfield.pin


    def initialize_actual(self):

        self.act_massflow = self.subfield.act_massflow / len(self.subfield.loops)
        self.act_tin = self.subfield.act_tin
        self.act_pin = self.subfield.act_pin
        self.act_tout = self.subfield.act_tout
        self.act_pout = self.subfield.act_pout

        self.massflow = self.act_massflow
        self.tin = self.act_tin
        self.pin = self.act_pin
        self.tout = self.act_tout
        self.pout = self.act_pout


    def initialize_rated(self):

        self.rated_massflow = self.subfield.rated_massflow / len(self.subfield.loops)
        self.rated_tin = self.subfield.rated_tin
        self.rated_pin = self.subfield.rated_pin
        self.rated_tout = self.subfield.rated_tout
        self.rated_pout = self.subfield.rated_pout

        self.massflow = self.rated_massflow
        self.tin = self.rated_tin
        self.pin = self.rated_pin
        self.tout = self.rated_tout
        self.pout = self.rated_pout


    def get_loop_avg_pr(self):

        pr_list = []
        for s in self.scas:
            for h in s.hces:
                pr_list.append(h.pr)

        return np.mean(pr_list)


    def load_from_prototype_loop(self, prototype_loop):

        self.massflow = prototype_loop.massflow
        self.tin = prototype_loop.tin
        self.act_tin = prototype_loop.act_tin
        self.pin = prototype_loop.pin
        self.act_pin = prototype_loop.act_pin
        self.tout = prototype_loop.tout
        self.pout = prototype_loop.pout
        self.act_tout = prototype_loop.act_tout
        self.act_pout = prototype_loop.act_pout
        self.pr_req_massflow = prototype_loop.pr_req_massflow
        self.pr_act_massflow = prototype_loop.pr_act_massflow
        self.req_massflow = prototype_loop.req_massflow
        self.act_massflow = prototype_loop.act_massflow


    def calc_loop_pr_for_massflow(self, row, solarpos, htf, model):

        # We work with the minimum massflow at night in order to
        # reduce thermal losses

        for s in self.scas:
            aoi = s.get_aoi(solarpos)
            for h in s.hces:
                model.calc_pr(h, htf, aoi, solarpos, row)

            self.tout = self.scas[-1].hces[-1].tout
            self.pout = self.scas[-1].hces[-1].pout

    def calc_loop_pr_for_tout(self, row, solarpos, htf, model):

        min_massflow = htf.get_massflow_from_Reynolds(
            self.solarfield.hce_settings['Absorber tube inner diameter'],
            self.tin, self.pin,
            self.solarfield.hce_settings['Min Reynolds'])

        max_error = 0.1  # % desviation tolerance
        search = True

        while search:

            self.calc_loop_pr_for_massflow(row, solarpos, htf, model)

            err = 100 * (abs(self.tout-self.solarfield.rated_tout) /
                   self.solarfield.rated_tout)

            if err > max_error:

                if self.tout >= self.solarfield.rated_tout:
                    self.massflow *= (1 + err / 100)
                    search = True
                elif (self.massflow > min_massflow and
                      self.massflow >
                      self.solarfield.solarfield_settings['min_massflow']):
                    self.massflow *= (1 - err / 100)
                    search = True
                else:
                    self.massflow = max(
                        min_massflow,
                        self.solarfield.solarfield_settings['min_massflow'])
                    self.calc_loop_pr_for_massflow(row, solarpos, htf, model)
                    search = False
            else:
                search = False

        self.req_massflow = self.massflow


    def show_parameter_vs_x(self, parameters = None):

        data = []

        for s in self.scas:
            for h in s.hces:
                data.append({'num': h.hce_order,
                             'tin': h.tin,
                             'tout': h.tout,
                             'pin': round(h.pin/100000,3),
                             'pout': round(h.pout/100000,3),
                             'pr': h.pr,
                             'qabs': h.qabs,
                             'qperd': h.qperd})

        loop_df = pd.DataFrame(data)

        if parameters:

            loop_df[parameters].plot(
                figsize=(20,10), linewidth=5, fontsize=20)
            plt.xlabel('HCE order', fontsize=20)

        else:

            loop_df[['num', 'tin', 'tout', 'pin', 'pout',
                     'pr', 'qabs', 'qperd']].plot(
                figsize=(20,10), linewidth=5, fontsize=20)

            plt.xlabel('HCE order', fontsize=20)


        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        # pd.set_option('display.max_colwidth', -1)
        print(loop_df[['num', 'tin', 'tout', 'pin', 'pout',
                       'pr', 'qabs', 'qperd']])
        #print(self.datasource.dataframe)

In [8]:
class PrototypeLoop(Loop):

    def __init__(self, solarfield):

        super().__init__(solarfield, 0)

        for s in range(self.solarfield.solarfield_settings['loop']['scas']):
            self.scas.append(SCA(self, s, self.solarfield.sca_settings))
            for h in range(self.solarfield.solarfield_settings['loop']['hces']):
                self.scas[-1].hces.append(
                    HCE(self.scas[-1], h, self.solarfield.hce_settings))


    def initialize(self, values = None):

        if values is not None:

            self.massflow = values['massflow']
            self.tin = values['tin']
            self.pin = values['pin']

        else:
            self.massflow = self.solarfield.massflow / self.solarfield.total_loops
            self.tin = self.solarfield.tin
            self.pin = self.solarfield.pin

### Clase Subfield

Un subcampo es un conjunto de lazos en los que se considera que el HTF se repartirá equitativamente. Cada subcampo dispone de una válvula de control de caudal a su entrada y representa el mayor grado de control de caudal de HTF que se puede alcanzar en el campo solar. Un campo solar suele contar con varios subcampos y cada uno de ellos, a su vez, cuenta con varios lazos.

La Clase Subfield mantiene referencias a lo lazos que lo constituyen, es deir, a cada una de las instancias que representan un lazo. También mantiene referencia al campo solar al que pertenece y cuenta con métodos para calcular cual será la tempertura de salida del HTF del subcampo solar una vez que el caudal de salida de cada lazo se haya mezclado con el del resto de lazos. Nótese que aunque se supone que todos los lazos del subcampo tienen el mismo caudal másico la temperatura de salida de cada uno de ellos puede ser diferente y, por tanto, la energía aportada por cada lazo también lo será.


In [4]:
class Subfield(object):
    '''
    Parabolic Trough Solar Field

    '''

    def __init__(self, solarfield, subfield_settings, loop_settings):

        self.solarfield = solarfield
        self.name = subfield_settings['name']
        self.parameters = subfield_settings
        self.loops = []

        self.tin = 0.0
        self.tout = 0.0
        self.pin = 0.0
        self.pout = 0.0
        self.tmax = 0.0
        self.massflow = 0.0
        self.req_massflow = 0.0

        self.act_tin = 0.0
        self.act_tout = 0.0
        self.act_pin = 0.0
        self.act_pout = 0.0
        self.act_massflow = 0.0


    def calc_massflow(self):

        mf = 0.0
        for l in self.loops:
            mf += l.massflow

        self.massflow = mf


    def calc_req_massflow(self):

        req_mf = 0.0
        for l in self.loops:
            req_mf += l.req_massflow

        self.req_massflow = req_mf

    def calc_pout(self):

        loops_var = []

        for l in self.loops:
            loops_var.append(l.pout * l.massflow)

        self.pout = np.mean(loops_var) / self.massflow

    def calc_tout(self, htf):
        '''
        Calculates HTF output temperature throughout the solar field as a
        weighted average based on the enthalpy of the mass flow in each
        loop of the solar field
        '''
        # H = 0.0
        # H2 = 0.0
        dH = 0.0
        dH_tmax = 0.0

        for l in self.loops:

            dH += l.massflow * htf.get_deltaH(l.tout, l.pout)
            dH_tmax += l.massflow * htf.get_deltaH(l.tmax, l.pout)

        dH /= self.massflow
        dH_tmax /= self.massflow
        self.tout =  htf.get_T(dH, self.pout)
        self.tmax = htf.get_T(dH_tmax, self.pout)


    def set_tin(self):

        self.tin = (self.solarfield.heatexchanger.htf_tout *
                    self.coolPipeLosses
                    )

    def apply_temp_limitation(self, htf):

        for l in self.loops:
            if l.tout > l.solarfield.tmax:
                HH = htf.get_deltaH(l.tout, l.pout)
                HL = htf.get_deltaH(l.solarfield.tmax, l.pout)
                l.tmax = l.solarfield.tmax
                l.wasted_power = l.massflow * (HH - HL)

            else:
                l.tmax = l.tout
                l.wasted_power = 0.0


    def loops_avg_out(self):

        tavg = 0.0
        cont = 0

        for l in self.loops:
            cont += 1
            tavg += l.tout

        return tavg / cont

    def load_rated(self):

        self.massflow = (self.solarfield.rated_massflow * len(self.loops) /
                         self.solarfield.total_loops)
        self.tin = self.solarfield.rated_tin
        self.pin = self.solarfield.rated_pin


    def load_actual(self, row):


        self.act_massflow = row[1][self.name+'.mf']
        self.act_tin = row[1][self.name+'.tin']
        self.act_pin = row[1][self.name+'.pin']
        self.act_tout = row[1][self.name+'.tout']
        self.act_pout = row[1][self.name+'.pout']


        self.massflow = self.act_massflow
        self.tin = self.act_tin
        self.pin = self.act_pin

### Clase Solarfield

El campo solar alberta el conjunto de de subcampos y, por tanto, todos los lazos de la instalación. Se trata del objeto que en última instancia queremos modelar con el fin de conocer cómo sera el comportamiento de la planta solar y su rendimento anual.
A la hora de definir el campo solar para el modelo es necesario conocer cuántos subcampos contiene, cuántos lazos hay en cada subcampo, qué configuración tiene cada lazo (número de SCAs en cada lazo y número de HCEs en cada SCA). También es importante conocer la distancia entre lazos con el fin de estimar el sombreado que se produce a primera y última hora del día.

El Clase Solarfield también recibe una serie de valores nominales para los que se ha realizado un diseño óptimo del sistema, como las temperaturas y presiones de entrada y salida del HTF, las temperaturas máximas y mínmas tolerables por razones de seguridad, el caudal nominal (normalmente se suele dar este caudal por lazo y el caudal  del campo solar será la suma de todos ellos), el caudal mínimo (existe cierta limitación tanto por la velocidad mínima de las bombas como por otras cuestiones operativas que desaconsejan que el HTF circule por debajo de este límite).

Cuando se crea una instancia de la Clase Solarfield, ésta recibe los datos de configuración que se ha pasado para la simulación y lanza el proceso de construcción que genera, en base a esa configuración, los diferentes subcampos, lazos, SCAs y HCEs que forma en campo solar.

In [5]:
class SolarField(object):
    '''
    Parabolic Trough Solar Field

    '''

    def __init__(self, solarfield_settings, sca_settings, hce_settings):


        self.solarfield_settings = solarfield_settings
        self.hce_settings = hce_settings
        self.sca_settings = sca_settings
        self.name = solarfield_settings['name']

        self.subfields = []
        self.total_loops = 0

        self.__build__()

        self.tin = 0.0
        self.tout = 0.0
        self.pin = 0.0
        self.pout = 0.0
        self.massflow = 0.0
        self.req_massflow = 0.0

        self.act_tin = 0.0
        self.act_tout = 0.0
        self.act_pin = 0.0
        self.act_pout = 0.0
        self.act_massflow = 0.0

        self.rated_tin = solarfield_settings['rated_tin']
        self.rated_tout = solarfield_settings['rated_tout']
        self.rated_pin = solarfield_settings['rated_pin']
        self.rated_pout = solarfield_settings['rated_pout']
        self.rated_massflow = (solarfield_settings['rated_massflow'] *
                               self.total_loops)
        self.min_massflow = (
            solarfield_settings['min_massflow'] * self.total_loops)

        self.tmax = solarfield_settings['tmax']
        self.tmin = solarfield_settings['tmin']

        # FUTURE WORK
        self.storage_available = False
        self.operation_mode = "subfield_heating"


    def __build__(self):

        for sf in self.solarfield_settings['subfields']:
            self.subfields.append(Subfield(self, sf, self.solarfield_settings))
            for l in range(sf['loops']):
                self.subfields[-1].loops.append(
                    Loop(self, l, self.subfields[-1]))
                for s in range(self.solarfield_settings['loop']['scas']):
                    self.subfields[-1].loops[-1].scas.append(
                        SCA(self.subfields[-1].loops[-1],
                            s,
                            self.sca_settings))
                    for h in range (self.solarfield_settings['loop']['hces']):
                        self.subfields[-1].loops[-1].scas[-1].hces.append(
                            HCE(self.subfields[-1].loops[-1].scas[-1],
                            h,
                            self.hce_settings))

        for sf in self.subfields:
            self.total_loops += len(sf.loops)


    def load_rated(self, htf):

        for sf in self.subfields:
                sf.load_rated()

    def initialize_rated(self):

        self.massflow = self.rated_massflow
        self.tin = self.rated_tin
        self.tout = self.rated_tout
        self.pin = self.rated_pin
        self.pout = self.rated_pout



    def load_actual(self, htf, row):

        massflow = 0.0
        H_tin = 0.0
        H_tout = 0.0
        list_pin = []
        list_pout = []

        for sf in self.subfields:
            sf.load_actual(row)
            massflow += sf.act_massflow
            H_tin += (sf.act_massflow *
                      htf.get_deltaH(sf.act_tin, sf.act_pin))
            H_tout += (sf.act_massflow *
                       htf.get_deltaH(sf.act_tout, sf.act_pout))
            list_pin.append(sf.act_pin * sf.act_massflow)
            list_pout.append(sf.act_pout * sf.act_massflow)

        H_tin /= massflow
        H_tout /= massflow

        self.act_massflow = massflow
        self.act_pin = np.mean(list_pin) / massflow
        self.act_pout = np.mean(list_pout) / massflow
        self.act_tin = htf.get_T(H_tin, self.act_pin)
        self.act_tout = htf.get_T(H_tout, self.act_pout)


    def initialize_actual(self):

        self.massflow = self.act_massflow
        self.tin = self.act_tin
        self.tout = self.act_tout
        self.pin = self.act_pin
        self.pout = self.act_pout


    def calc_massflow(self):

        mf = 0.0
        req_mf = 0.0

        for sf in self.subfields:
            mf += sf.massflow
            req_mf += sf.req_massflow

        self.massflow = mf
        self.req_massflow = req_mf


    def calc_req_massflow(self):

        mf = 0.0

        for sf in self.subfields:
            mf += sf.req_massflow

        self.req_massflow = mf

    def calc_pout(self):

        subfields_var = []

        for sf in self.subfields:
            subfields_var.append(sf.pout * sf.massflow)

        self.pout = np.mean(subfields_var) / self.massflow

    def calc_tout(self, htf):
        '''
        Calculates HTF output temperature throughout the solar plant as a
        weighted average based on the enthalpy of the mass flow in each
        loop of the solar field
        '''

        dH = 0.0
        dH_tmax = 0.0

        for sf in self.subfields:

            dH += sf.massflow * htf.get_deltaH(sf.tout, sf.pout)
            dH_tmax += sf.massflow * htf.get_deltaH(sf.tmax, sf.pout)

        dH /= self.massflow
        dH_tmax /= self.massflow
        self.tout = htf.get_T(dH, self.pout)
        self.tmax = htf.get_T(dH_tmax, self.pout)

    def set_act_tout(self, htf):
        '''
        Calculates HTF output temperature throughout the solar plant as a
        weighted average based on the enthalpy of the mass flow in each
        loop of the solar field
        '''

        dH_actual = 0.0

        for sf in self.subfields:

            dH_actual += sf.act_massflow * htf.get_deltaH(sf.tout, sf.pout)
            dH_actual += sf.act_massflow * htf.get_deltaH(sf.act_tout, sf.act_pout)

        dH_actual /= self.act_massflow
        self.act_tout = htf.get_T(dH_actual, self.act_pout)


    def set_tin(self, htf):
        '''
        Calculates HTF output temperature throughout the solar plant as a
        weighted average based on the enthalpy of the mass flow in each
        loop of the solar field
        '''
        H = 0.0

        for sf in self.subfields:

            H += (htf.get_deltaH(sf.tin, sf.pin) * sf.massflow)

        H /= self.massflow
        self.tin = htf.get_T(H, self.rated_pin)


    def set_pin(self):

        subfields_var = []

        for sf in self.subfields:
            subfields_var.append(sf.pin * sf.massflow)

        self.pin = np.mean(subfields_var) / self.massflow


    def set_act_pin(self):

        subfields_var = []

        for sf in self.subfields:
            subfields_var.append(sf.act_pin * sf.act_massflow)

        self.act_pin = np.mean(subfields_var) / self.act_massflow


    def get_thermalpoweroutput(self, htf):

        HL = htf.get_deltaH(self.tin, self.pin)
        HH = htf.get_deltaH(self.tout, self.pout)

        return (HH - HL) * self.massflow



    # def calcRequired_massflow(self):
    #     req_massflow = 0
    #     for sf in self.subfields:
    #         req_massflow += sf.calc_required_massflow()
    #     self.req_massflow = req_massflow

    def set_operation_mode(self, mode = None):

        if mode is not None:
            self.operation_mode = mode
        else:
            if self.storage_available == True:
                pass
            else:
                if self.tout > self.tin:
                   self.operation_mode = "solarfield_heating"
                else:
                    self.operation_mode = "solarfield_not_heating"



    def print(self):

        for sf in self.subfields:
            for l in sf.loops:
                for s in l.scas:
                    for h in s.hces:
                        print("subfield: ", sf.name,
                              "Lazo: ",l.loop_order,
                              "SCA: ", s.sca_order,
                              "HCE: ", h.hce_order,
                              "tin", "=", h.tin,
                              "tout", "=", h.tout)

### Clase Fluid y sus clases hijas FluidCoolProp y FluidTabular

Para modelar el HTF (Heat Transfer Fluid) se ha creado una Clase Fluid. Las propiedades del HTF pueden obtenerse mediante funciones polinomicas con coeficientes constantes calculados experimentalmente o desde librerias preexistentes como CoolProp. Por ese motivo, según se opte por un método u otro, se han creado las subclases FluidCoolProp y FluidTabular. No obstante, se ha comprobado que el número de tipos de HTF que existen en la librería FluidCoolProp no es muy grande, limitándose a Therminol VP-1, Syltherm 800. No se encuentra en esta librería el aceite Dowtherm A, que es el que se emplea en la planta cuyos datos se han empleado para el desarrollo de esta herramienta. Pero el mayor inconveniente reside en que CoolProp devuelve valores solo dentro del rango de temperaturas de uso válidas según el fabricante. Este rango es demasiado estricto y se producen problemas debido a la devolución de valores no numéricos, especialmente cuando se está calculando la temperatura teórica de salida del HTF cuando hay sobrecalentamiento. Por este motivo se ha hecho uso mayoritariamente de la clase FluidTabular, con funciones polinómicas con coeficientes calculados a partir de los datos ofrecidos por los fabricantes.

La clase Fluid sus clases hijas ofrecen métodos para calcular la densidad, viscosidad cinemática, número de Reynolds, calor específico, conductividad térmica y entalpia en función de la temperatura y la presión. 
También ofrecen un método para calcular la temperatura del fludo en función de la entalpía y la presión, considerando entalpía cero para líquido saturado según ASHRAE a una tempratura de 285.856 K.




In [6]:
class Fluid(object):

    def test_fluid(self, tmax, tmin, p):

        data = []

        for tt in range(int(round(tmax)), int(round(tmin)), -5):
            data.append({'T': tt,
                         'P': p,
                         'cp': self.get_cp(tt, p),
                         'rho': self.get_density(tt, p),
                         'mu': self.get_dynamic_viscosity(tt, p),
                         'kt': self.get_thermal_conductivity(tt, p),
                         'H': self.get_deltaH(tt, p),
                         'T-H': self.get_T(self.get_deltaH(tt, p), p)})
        df = pd.DataFrame(data)
        print(round(df, 6))



In [9]:
class FluidCoolProp(Fluid):

    def __init__(self, settings = None):

        self.name = settings['name']
        self.tmax = settings['tmax']
        self.tmin = settings['tmin']
        self.coolpropID = settings['CoolPropID']

    def get_density(self, t, p):

        if t > self.tmax:
            t = self.tmax

        return PropsSI('D','T',t,'P', p, self.coolpropID)

    def get_dynamic_viscosity(self, t, p):

        if t > self.tmax:
            t = self.tmax

        return  PropsSI('V','T',t,'P', p, self.coolpropID)

    def get_cp(self, t, p):

        if t > self.tmax:
            t = self.tmax
        return PropsSI('C','T',t,'P', p, self.coolpropID)

    def get_thermal_conductivity(self, t, p):
        ''' Saturated Fluid conductivity at temperature t '''

        if t > self.tmax:
            t = self.tmax

        return PropsSI('L','T',t,'P', p, self.coolpropID)

    def get_deltaH(self, t, p):

        if t > self.tmax:
            t = self.tmax

        CP.set_reference_state(self.coolpropID,'ASHRAE')

        deltaH = PropsSI('H','T',t ,'P', p, self.coolpropID)

        CP.set_reference_state(self.coolpropID, 'DEF')

        return deltaH

    def get_T(self, h, p):

        # if t > self.tmax:
        #     t = self.tmax

        CP.set_reference_state(self.coolpropID,'ASHRAE')
        temperature = PropsSI('T', 'H', h, 'P', p, self.coolpropID)
        CP.set_reference_state(self.coolpropID, 'DEF')

        return temperature


    def get_Reynolds(self, dri, t, p, massflow):

        if t > self.tmax:
            t = self.tmax

        return massflow * np.pi * (dri**3) /( 4 * self.get_density(t, p))

    def get_massflow_from_Reynolds(self, dri, t, p, re):

        if t > self.tmax:
            t = self.tmax

        return re * np.pi * dri * self.get_dynamic_viscosity(t, p) / 4

In [10]:
class FluidTabular(Fluid):

    def __init__(self, settings=None):

        self.name = settings['name']
        self.cp = settings['cp']
        self.rho = settings['rho']
        self.mu = settings['mu']
        self.kt = settings['kt']
        self.h = settings['h']
        self.t = settings['t']
        self.tmax = settings['tmax']
        self.tmin = settings['tmin']

        self.cp += [0.] * (6 - len(self.cp))
        self.rho += [0.] * (6 - len(self.rho))
        self.mu += [0.] * (6 - len(self.mu))
        self.kt += [0.] * (6 - len(self.kt))


    def get_density(self, t, p):

        # Dowtherm A.pdf, 2.2 Single Phase Liquid Properties. pg. 8.

        rho0, rho1, rho2, rho3, rho4, rho5 = tuple(self.rho)

        return (rho0 + rho1 * t + rho2 * t**2 + rho3 * t**3 +
                rho4 * t**4 + rho5 * t**5) * (p* 1.0e-4)**1.0e-3

    def get_dynamic_viscosity(self, t, p):

        mu0, mu1, mu2, mu3, mu4, mu5 = tuple(self.mu)

        if t > self.tmax:
            t= self.tmax

        return (mu0 + mu1 * t + mu2 * t**2 + mu3 * t**3 +
                mu4 * t**4 + mu5 * t**5)

    def get_cp(self, t, p):

        cp0, cp1, cp2, cp3, cp4, cp5 = tuple(self.cp)

        return (cp0 + cp1 * t + cp2 * t**2 + cp3 * t**3 +
                cp4 * t**4 + cp5 * t**5)

    def get_thermal_conductivity(self, t, p):
        ''' Saturated Fluid conductivity at temperature t '''

        kt0, kt1, kt2, kt3, kt4, kt5 = tuple(self.kt)

        kt =(kt0 + kt1 * t + kt2 * t**2 + kt3 * t**3 +
                kt4 * t**4 + kt5 * t**5)

        if kt < 0:
            kt = 0

        return kt


    def get_deltaH(self, t, p):

        h0, h1, h2, h3, h4, h5 = tuple(self.h)

        href =(h0 + h1 * _T_REF + h2 * _T_REF**2 + h3 * _T_REF**3 +
               h4 * _T_REF**4 + h5 * _T_REF**5)

        h = (h0 + h1 * t + h2 * t**2 + h3 * t**3 + h4 * t**4 + h5 * t**5)
        return (h - href)

    def get_T(self, h, p):

        t0, t1, t2, t3, t4, t5 = tuple(self.t)

        return (t0 + t1 * h + t2 * h**2 + t3 * h**3 +
                t4 * h**4 + t5 * h**5)

    def get_Reynolds(self, dri, t, p, massflow):

        if t > self.tmax:
            t= self.tmax

        return (4 * massflow /
                (np.pi * dri * self.get_dynamic_viscosity(t,p)))

    def get_massflow_from_Reynolds(self, dri, t, p, re):

        if t > self.tmax:
            t = self.tmax

        return re * np.pi * dri * self.get_dynamic_viscosity(t,p) / 4


### Clase Model y Subclases ModelBarbero4grade

Se emplean clases derivadas de la Clase Model (en realidad un mera clase interfaz), para implementar los diferentes modelos empleados para calcular el rendimiento y para simular, por tanto, el funcionamiento de cada HCE. Es en cada una de estas clases donde se desarrolla el algoritmo que, a partir de los parámetros que definen al HCE (longitud, absorptancia, emisividad...), las variables que definen el estado del HTF que circula por él (temperatura y presión de entrada, caudal másico) y de las condiciones de operación (radiación solar, ángulo de incidencia, temperatura ambiente, velocidad del viento...), así como de otros factores que detallarán más adelante, resuelve las ecuaciones definidas en el modelo y nos permite conocer las condiciones del HTF a la salida de cada HCE.

El principal modelo empleado es el modelo de 4º orden desarrollado en <a id="ref-1" href="#cite-Barbero2016">1</a>






In [2]:
class Model(object):

    def __init__(self):
        pass
    
class ModelBarbero4grade(Model):


    def __ini__(self):
        super(Model, self).__init__()


    @classmethod
    def calc_pr(cls, hce, htf, aoi, solarpos, row):

        flag_0 = datetime.now()

        pressure = hce.sca.loop.pin
        hce.set_tin()
        hce.set_pin()
        hce.tout = hce.tin
        tin = hce.tin
        tf = hce.tin
        tro = hce.tin
        tri = hce.tin

        massflow = hce.sca.loop.massflow

        dni = row[1]['DNI']
        wspd = row[1]['Wspd']
        text = row[1]['DryBulb']


        krec = 0.0153 * (tin - 273.15) + 14.77
        sigma = sc.constants.sigma
        dro = hce.parameters['Absorber tube outer diameter']
        dri = hce.parameters['Absorber tube inner diameter']
        dgo = hce.parameters['Glass envelope outer diameter']
        dgi = hce.parameters['Glass envelope inner diameter']
        L = hce.parameters['Length']
        A = hce.sca.parameters['Aperture']
        x = 1 #  Calculation grid fits hce longitude
        IAM = hce.sca.get_IAM(aoi)
        pr_opt_peak = hce.get_pr_opt_peak(aoi, solarpos, row)
        pr_geo = hce.get_pr_geo(aoi, solarpos, row)
        pr_shadows = hce.get_pr_shadows(aoi, solarpos, row)
        # pr_shadows = 0.5

        cg = A /(np.pi*dro)

        #  nu_air = cinematic viscosity PROVISIONAL A FALTA DE VALIDAR TABLA
        # tabla en K
        nu_air = Air.get_cinematic_viscosity(text)
        wspd = row[1]['Wspd']

        #  Reynols number for wind at
        reext = dgo * wspd / nu_air

        #hext, eext = cls.get_hext_eext(hce, reext, tro, wspd)
        eext = hce.get_emittance(tro, wspd)
        hext = hce.get_hext(wspd)

        cp = htf.get_cp(tf, hce.pin)
        cpri = cp

        # Ec. 4.14
        mu = htf.get_dynamic_viscosity(tf, pressure)
        muri = mu

        rho = htf.get_density(tf, pressure)
        rhori = rho

        kf = htf.get_thermal_conductivity(tf, pressure)
        kfpri = kf

        alpha = kf / (rho * cp)
        alphari = alpha

        prf = mu / alpha  #  Prandtl = viscosidad dinámica / difusividad_termica
        prfri = prf

        #  Correlación de Gnielinski (utilizado por Forristall [1]
        #  y verificado en [38]) según ec. 4.15
        redri = 4 * massflow / (mu * np.pi * dri)  # Reynolds
        cf = (1.58 * np.log(redri) - 3.28)**-2

        # Número de Nusselt alto, casi independiente inicialmente de tri
        # ya que estamos considerando tri = tf o casi igual

        nug = ((cf / 2) * (redri - 1000) * prf * (prf / prfri)**0.11 /
               (1 + 12.7 * (cf / 2)**0.5 * (prf**(2 / 3) - 1)))


        #  Internal transmission coefficient.
        hint = kf * nug / dri

        #  Ec. 3.23
        qperd = sigma * eext * (tro**4 - text**4) + hext * (tro - text)

        # Ec. 3.50 Barbero
        qcrit = qperd

        #  Ec. 3.51 Barbero
        ucrit = 4 * sigma * eext * tin**3 + hext
        # Ec. 3.22
        urec = 1 / ((1 / hint) + (dro * np.log(dro / dri)) / (2 * krec))
        #  Ec. 3.30
        NTU = urec * x * L * sc.pi * dro / (massflow * cp)

        #  Ec. 3.20 Barbero
        qabs = (pr_opt_peak * IAM * cg * dni * pr_geo * pr_shadows)

        if qabs > 0:
            pr0 = (1 - qcrit / qabs)/(1 + ucrit / urec)

            if pr0 <0:
                pr0 = 0.0
        else:
            pr0 = 0.0

        pr1 = pr0
        tro1 = tf + qabs * pr1 / urec

        if qabs > qcrit:
            errtro = 1.
            errpr = 1.
            step = 0

            while (errtro > 0.1 or errpr > 0.01):

                step += 1
                flag_1 = datetime.now()

                f0 = qabs/(urec*(tin-text))
                f1 = ((4*sigma*eext*text**3)+hext)/urec
                f2 = 6*(text**2)*(sigma*eext/urec)*(qabs/urec)
                f3 = 4*text*(sigma*eext/urec)*((qabs/urec)**2)
                f4 = (sigma*eext/urec)*((qabs/urec)**3)

                fx = lambda pr0: (1 - pr0 -
                                  f1 * (pr0 + (1 / f0)) -
                                  f2 * ((pr0 + (1 / f0))**2) -
                                  f3 * ((pr0 + (1 / f0))**3) -
                                  f4 * ((pr0 + (1 / f0))**4))

                dfx = lambda pr0: (-1 - f1 -
                                   2 * f2 * (pr0 + (1 / f0)) -
                                   3 * f3 * (pr0 + (1 / f0))**2 -
                                   4 * f4 * (pr0 + (1 / f0))**3)

                root = sc.optimize.newton(fx,
                                          pr0,
                                          fprime=dfx,
                                          maxiter=100000)

                pr0 = root
                z = pr0 + (1/f0)
                g1 = 1+f1+2*f2*z+3*f3*z**2+4*f4*z**3
                g2 = 2*f2+6*f3*z+12*f4*z**2
                g3 = 6*f3+24*f4*z

                pr2 = ((pr0*g1/(1-g1))*(1/(NTU*x)) *
                       (sc.exp((1-g1)*NTU*x/g1)-1) -
                       (g2/(6*g1))*(pr0*NTU*x)**2 -
                       (g3/(24*g1)*(pr0*NTU*x)**3))

                errpr = abs(pr2-pr1)
                pr1 = pr2
                hce.pr = pr1
                hce.qabs = qabs
                hce.set_tout(htf)
                hce.set_pout(htf)
                tf = 0.5 * (hce.tin + hce.tout)

                tri = tro1

                cp = htf.get_cp(tf, pressure)
                cpri = htf.get_cp(tri, pressure)

                mu = htf.get_dynamic_viscosity(tf, pressure)
                muri = htf.get_dynamic_viscosity(tri, pressure)

                rho = htf.get_density(tf, pressure)
                rhori =  htf.get_density(tri, pressure)


                kf = htf.get_thermal_conductivity(tf, pressure)
                kfpri = htf.get_thermal_conductivity(tri, pressure)

                #  alpha : difusividad térmica
                alpha = kf / (rho * cp)
                alphari = kfpri / (rhori * cpri)

                prf = mu / alpha  #  Prandtl = viscosidad dinámica / difusividad_termica
                prfri =  muri / alphari

                redri = 4 * massflow / (mu * np.pi * dri)  # Reynolds

                # Número de Nusselt alto, casi independiente inicialmente de tri
                # ya que estamos considerando tri = tf o casi igual
                nug = ((cf / 2) * (redri - 1000) * prf * (prf / prfri)**0.11 /
                       (1 + 12.7 * (cf / 2)**0.5 * (prf**(2 / 3) - 1)))

                #nudb = 0.023 * redri**0.8 * prf**0.4
                hint = kf * nug / dri
                # hext, eext = cls.get_hext_eext(hce, reext, tro1, wspd)
                eext = hce.get_emittance(tro1, wspd)
                hext = hce.get_hext(wspd)

                trec = (tro1+tri)/2
                # Ec. 4.22
                hce.set_krec(trec)
                krec = hce.parameters['krec']
                # Ec. 3.22
                urec = 1 / ((1 / hint) + (dro * np.log(dro / dri)) / (2 * krec))
                NTU = urec * x * L * sc.pi * dro / (massflow * cp)  # Ec. 3.30

                tro2 = tf+qabs*pr2/urec
                errtro = abs(tro2-tro1)
                tro1 = tro2
                qperd = sigma * eext * (tro1**4 - text**4) + hext * (tro1 - text)

                flag_2 = datetime.now()

        else:
            hce.pr = pr1
            hce.qperd = qperd
            hce.qabs = qabs
            hce.set_tout(htf)

            #FLUJO LAMINAR
            hce.set_pout(htf)
            flag_3 = datetime.now()


    @classmethod
    def get_hext_eext(cls, hce, reext, tro, wind):

        eext = 0.
        hext = 0.

        if hce.parameters['emi'] == 'Solel UVAC 2/2008':
            pass

        elif hce.parameters['Name'] == 'Solel UVAC 3/2010':
            pass

        elif hce.parameters['Name'] == 'Schott PTR70':
            pas

        if (hce.parameters['coating'] == 'CERMET' and
            hce.parameters['annulus'] == 'VACUUM'):
            if wind > 0:
                eext = 1.69E-4*reext**0.0395*tro+1/(11.72+3.45E-6*reext)
                hext = 0.
            else:
                eext = 2.44E-4*tro+0.0832
                hext = 0.
        elif (hce.parameters['coating'] == 'CERMET' and
              hce.parameters['annulus'] == 'NOVACUUM'):
            if wind > 0:
                eext = ((4.88E-10 * reext**0.0395 + 2.13E-4) * tro +
                        1 / (-36 - 1.29E-4 * reext) + 0.0962)
                hext = 2.34 * reext**0.0646
            else:
                eext = 1.97E-4 * tro + 0.0859
                hext = 3.65
        elif (hce.parameters['coating'] == 'BLACK CHROME' and
              hce.parameters['annulus'] == 'VACUUM'):
            if wind > 0:
                eext = (2.53E-4 * reext**0.0614 * tro +
                        1 / (9.92 + 1.5E-5 * reext))
                hext = 0.
            else:
                eext = 4.66E-4 * tro + 0.0903
                hext = 0.
        elif (hce.parameters['coating'] == 'BLACK CHROME' and
              hce.parameters['annulus'] == 'NOVACUUM'):
            if wind > 0:
                eext = ((4.33E-10 * reext + 3.46E-4) * tro +
                        1 / (-20.5 - 6.32E-4 * reext) + 0.149)
                hext = 2.77 * reext**0.043
            else:
                eext = 3.58E-4 * tro + 0.115
                hext = 3.6

        return hext, eext

## References

<a id="cite-Barbero2016"/><sup><a href=#ref-1>[^]</a></sup>Rubén Barbero, Antonio Rovira, María José Montes, José María Martínez Val. 2016. _A new approach for the prediction of thermal efficiency in solar receivers_.

